In [ ]:
# import os

# # Fix to be able to import python modules inside a notebook
# os.chdir("..")

# Good code

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

**Как передавать параметры?**

0. CAPS_TEXT
1. argparse
2. click (модный argparse)
3. Hydra (для сложных конфигов)

In [ ]:
DATA_PATH = "data/train.csv"
TEST_DATA_PATH = "data/test.csv"

In [ ]:
# Data loading
df = pd.read_csv(DATA_PATH)

In [ ]:
def clean_df(df):
    # Drop NaN, replace columns e.t.c 
#     features[(features["budget"] > 0) & (features["popularity"] > 0)]
#     features.dropna(subset=["budget", "popularity"], inplace=True)

# 
    return df

def train_val_test_split(
    df: pd.DataFrame, train_pct: float = 0.6, val_pct:float = 0.2, test_pct: float = 0.2):
    
    N = len(df)
    # Логика для разбиения
#     ...
    train_df, val_df, test_df = df[:1000], df[1000:1200], df[1200:1400]
    return train_df, val_df, test_df

# Clean
df = clean_df(df)

# Split
train_df, val_df, test_df = train_val_test_split(df)

In [ ]:
df.sample(2)[['budget', ]]

In [ ]:
# EDA - Explanotary Data Analyses
# TODO

# from sklearn import StandardScaler

def preprocess_df(df, columns=('budget','popularity','revenue'), stats=None):
    # Select columns
    
    df = df[list(columns)]
    if stats is None:
        stats = {
            'mean': df.mean(),
            'std': df.std(),
        }
        # Считаем распределния и нормализуем по данным из DF
        # Standard Scaling
#          .loc[row_indexer,col_indexer] = value instead
        df['budget'] = ((df['budget'] - stats['mean']['budget']) /  stats['std']['budget'])
        # features['popularity'] = ((features.popularity - features.popularity.mean()) /  features.popularity.std())
        # features['revenue'] = ((features.revenue - features.revenue.mean()) /  features.revenue.std())
        # features.sample(10)
    else:
        # Используем stats для получения среднего и среднеквадратичного отклонения
        pass
  

    return df, stats


train_df, stats = preprocess_df(train_df)
(val_df, _), (test_df, _) = preprocess_df(val_df, stats=stats), preprocess_df(test_df, stats=stats)

In [ ]:

# MSE loss function
def mse_loss(x: np.ndarray, y, np.ndarray):
    squared_error = (x - y) ** 2
    sum_squared_error = np.sum(squared_error)
    loss = sum_squared_error / val_true.size
    return loss



In [ ]:
train_df = pd.read_csv(TRAIN_CSV_PATH)
# train_df['crew'].unique()
train_df.columns
features = train_df[["budget", "popularity", 'revenue']]
features.columns
features = features[(features["budget"] > 0) & (features["popularity"] > 0)]

# features.dropna(subset=[], inplace=True)

# Garbage code

In [ ]:
def calculate_model_output(w, b, x):
    m = x.shape  # the number of training examples
    f_wb = np.zeros(m)
    for i in range(len(x)):
        f_wb[i] = w * x[i] + b
    return f_wb


def train_one_step(X_train, y_train, w, learning_rate):
    f = X_train.dot(w)
    err = f - y_train
    grad = 2 * X.T.dot(err) / len(X_train)
    w -= LR * grad
    if i % 10 == 0:
        x_plot.append(i)
        y_plot.append(err.mean())
        return err.mean

def val_one_step(X_val, y_val, w):
    y_pred = X_val.dot(w)
    err = mse_loss(y_pred, y_val)
    err


loss_val = 

In [ ]:
features.sample(10)
# standard scaling
features['budget'] = ((features.budget - features.budget.mean()) /  features.budget.std())
features['popularity'] = ((features.popularity - features.popularity.mean()) /  features.popularity.std())
features['revenue'] = ((features.revenue - features.revenue.mean()) /  features.revenue.std())
features.sample(10)

In [ ]:
features.describe()

w1 = 0.00000001
w2 = 100
w3 = 1
# 
train_x = features.to_numpy()
 
target = train_df["revenue"]
target
import numpy as np
import matplotlib.pyplot as plt

# Plotting
x_vals = np.arange(-20, 20, 0.01)
y_vals = np.square(x_vals)




In [ ]:
# TODO Do random sampling!

import random 
import numpy as np
# Fix random seed!

len_train = int(len(features) / 100 * 60)
len_val = int(len(features) / 100 * 80)
len_test = len(features) - len_train - len_val

train_features = train_x[:len1]
val_features = train_x[len1:len2]
test_features = train_x[len2:]

len(features), len(train_features), len(val_features), len(test_features)

In [ ]:
features.budget.hist(bins=20, )
features.popularity.hist(bins=20, )
features[features.popularity > 50]
features.popularity.min(), features.popularity.max()
((features.popularity - features.popularity.min()) / (features.popularity.max() - features.popularity.min())).describe()
features.popularity.describe()



In [ ]:
# Min / max scaling
((features.budget - features.budget.min()) / (features.budget.max() - features.budget.min())).describe()
# standard scaling
((features.budget - features.budget.mean()) /  features.budget.std()).describe()
train_df[train_df.popularity > 60][['original_title', 'popularity']]
features

In [ ]:
w = np.random.randn(3)

In [ ]:
plt.plot(x_vals, y_vals, "blue")
plt.plot(x_vals, y_vals, "blue")
plt.grid(True, which="major")
plt.show()
x_train = features[['budget', 'popularity']]
y_train = features['revenue'].to_numpy()
X = np.concatenate([x_train.to_numpy(), np.ones([len(x_train), 1])], axis=1)
features.to_numpy().shape, np.ones([len(features), 1]).shape
X @ w
X, y_train


In [ ]:
# TODO Переименовать переменные!

In [ ]:
x_train1 = features[['budget', 'popularity']][:2100]
y_train1 = features['revenue'].to_numpy()[:2100]
X1 = np.concatenate([x_train1.to_numpy(), np.ones([len(x_train1), 1])], axis=1)
# X1 @ w
X1, y_train1

In [ ]:
x_train2 = features[['budget', 'popularity']][2100:2600]
y_train2 = features['revenue'].to_numpy()[2100:2600]
X2 = np.concatenate([x_train2.to_numpy(), np.ones([len(x_train2), 1])], axis=1)
# X2 @ w
# X2, y_train2


x_train3 = features[['budget', 'popularity']][2601:]
y_train3 = features['revenue'].to_numpy()[2601:]
X3 = np.concatenate([x_train3.to_numpy(), np.ones([len(x_train3), 1])], axis=1)
# X3 @ w
# X3, y_train3


# w = random_normal()             # можно пробовать и другие виды инициализации
# repeat S times:                 # другой вариант: while abs(err) > tolerance
#    f = X.dot(w)                 # посчитать предсказание
#    err = f - y                  # посчитать ошибку
#    grad = 2 * X.T.dot(err) / N  # посчитать градиент
#    w -= alpha * grad            # обновить веса

In [ ]:
import func_ds
LR = 0.01
w = np.random.randn(3)
N = len(y_train)

func_ds.x_plot = []
func_ds.y_plot = []
#  traing dataset
for i in range(3):
    print(w)
    func_ds.train_one_step(X1, w, y_train1, N, LR, i)
        
# print(func_ds.x_plot)
# print(func_ds.y_plot)

In [ ]:


fig, ax = plt.subplots()

ax.plot(func_ds.x_plot, func_ds.y_plot)
ax.set(title='График ошибки функции на training датасете')
plt.xlabel("цикл обучения")
plt.grid(True, which="major")
plt.ylabel("квадрат ошибки")
plt.show()

In [ ]:
def change_list(a):
    x = a.pop()
    print(x)

# Side-artefacts 
A = [1, 2, 3, 4]
print(A)
change_list(A)
print(A)

In [ ]:
w = np.random.randn(3)

func_ds.x_plot = []
func_ds.y_plot = []
#  traing dataset
for i in range(2):
    print(w)
    func_ds.train_one_step(X2, w, y_train2, N, LR, i)

In [ ]:
fig, ax = plt.subplots()

ax.plot(func_ds.x_plot, func_ds.y_plot, "red")
ax.set(title='График ошибки функции на value датасете')
plt.grid(True, which="major")
plt.xlabel("цикл обучения")
plt.ylabel("квадрат ошибки")
plt.show()

In [ ]:
func_ds.x_plot = []
func_ds.y_plot = []
#  traing dataset
for i in range(1000):
    func_ds.train_one_step(X3, w, y_train3, N, LR, i)

In [ ]:
fig, ax = plt.subplots()

print(func_ds.y_plot)

ax.plot(func_ds.x_plot, func_ds.y_plot)
ax.set(title='График ошибки функции на test датасете')
plt.grid(True, which="major")
plt.xlabel("цикл обучения")
plt.ylabel("квадрат ошибки")
plt.show()